# Code to demo 3D Cell Generation API

This code uses the [hra-3d-cell-generation-api](https://github.com/cns-iu/hra-3d-cell-generation-api) to get cell summaries for anatomical structures to be displayed in [ra-cell-type-populations-visualization](https://github.com/cns-iu/hra-cell-type-populations-visualization), which is in turn displayed on the [companion website](https://cns-iu.github.io/hra-cell-type-populations-supporting-information/). The code is also used for one of the figures.

## Import statements

In [69]:
import requests
import json
import csv
import pandas as pd
from io import StringIO

## Generate request params

In [70]:
# Choose organ and AS to get 3D cells for
# organ = "https://ccf-ontology.hubmapconsortium.org/objects/v1.2/VH_M_Kidney_L.glb"
organ = "https://cdn.humanatlas.io/digital-objects/ref-organ/heart-female/v1.3/assets/3d-vh-f-heart.glb"
as_name = "VH_F_left_ventricle"
max_cells = 6000

# Set request params
request_params = {
    "file": organ,
    "file_subpath": as_name,
    "num_nodes": max_cells,
    "node_distribution": {}
}

### Get AS-CT counts from HRA LOD

In [71]:
# Get cell type distribution from HRA LOD
url = 'https://grlc.io/api-git/hubmapconsortium/ccf-grlc/subdir/hra-pop//cell_types_in_anatomical_structurescts_per_as?endpoint=https%3A%2F%2Flod.humanatlas.io%2Fsparql'
header = {"accept": "text/csv"}

# make the GET request and capture text
response = requests.get(url, headers=header)
csv_response = response.text

# convert to data frame
df = pd.read_csv(StringIO(csv_response))

### Create params for hra-3d-cell-generation-api as JSON

In [72]:
# Subset to only get cells for the desired anatomical structure
subset_df = df[((df['as_label'] == 'outer cortex of kidney')) &
               (df['organ'] == 'Heart') &
               (df['sex'] == 'Female') &
               (df['tool'] == 'azimuth')
               ]

# Extract cell labels and percentages for key-value pairs in JSON params
cell_information = subset_df.loc[:, ['cell_label', 'cell_percentage']]

# Create a dictionary where cell_label is the key and cell_percentage is the value
cell_label = cell_information['cell_label'].tolist()
cell_percentage = cell_information['cell_percentage'].tolist()
pairs = dict(zip(cell_label, cell_percentage))

### Add to JSON params

In [73]:
request_params['node_distribution'] = pairs

## Fetch 3D cell positions from API and export them to CSV

In [77]:
# set the URL
url = 'https://apps.humanatlas.io/api/v1/mesh-3d-cell-population'

# make the POST request 
request = requests.post(url, json=request_params)
response = request.text.split('\n')

print(f'request.text:\n {request.text}')

# export to CSV
with open('output/3d_cell_distribution.csv', 'w', newline='') as csvfile:
    writer = csv.writer(csvfile, delimiter=',')
    for line in response:
        writer.writerow(line.split(','))

request.text:
 x,y,z,Cell Type
